# **hkl_soleil** E4CH

The IUCr provides a schematic of the 4-circle
[diffractometer](http://ww1.iucr.org/iucr-top/comm/cteach/pamphlets/2/node14.html)
(in horizontal geometry typical of a laboratory instrument).  In **hklpy2**,
this is the *E4CH* {ref}`geometry <geometries-hkl_soleil-e4ch>`.

<!-- image source:
  http://ww1.iucr.org/iucr-top/comm/cteach/pamphlets/2/
  -->
![E4CH geometry](../_static/img69-iucr-4-circle.gif)

**Note**: At X-ray synchrotrons, the vertical *E4CV* geometry is more common
due to the polarization of the X-rays.

## Setup the *E4CH* diffractometer in **hklpy2**

The *hkl_soleil* *E4CH* [geometry](https://people.debian.org/~picca/hkl/hkl.html)
is described:

axis  | moves    | rotation axis    | vector
---   | ---      | ---              | ---
omega | sample   | {math}`\vec{z}`  | `[0 0 1]`
chi   | sample   | {math}`\vec{x}`  | `[1 0 0]`
phi   | sample   | {math}`\vec{z}`  | `[0 0 1]`
tth   | detector | {math}`\vec{z}`  | `[0 0 1]`

* xrays incident on the {math}`\vec{x}`   direction (1, 0, 0)

## Define _this_ diffractometer

Use the **hklpy2** `creator()` function to create a diffractometer
object.  The diffractometer object will have simulated rotational axes.

We'll provide the geometry and solver names.
By convention, the `name` keyword is the same as the object name.

See the [geometry tables](../diffractometers.rst) for
a more complete description of the available diffractometers.

Create the Python diffractometer object (`fourc`).

In [1]:
import hklpy2

fourc = hklpy2.creator(name="fourc", geometry="E4CH", solver="hkl_soleil")

## Add a sample with a crystal structure

In [2]:
from hklpy2.user import add_sample, calc_UB, cahkl, cahkl_table, pa, set_diffractometer, setor, wh

set_diffractometer(fourc)
add_sample("silicon", a=hklpy2.SI_LATTICE_PARAMETER)

Sample(name='silicon', lattice=Lattice(a=5.431, system='cubic'))

## Setup the UB orientation matrix using *hklpy*

Define the crystal's orientation on the diffractometer using 
the 2-reflection method described by [Busing & Levy, Acta Cryst 22 (1967) 457](https://www.psi.ch/sites/default/files/import/sinq/zebra/PracticalsEN/1967-Busing-Levy-3-4-circle-Acta22.pdf).

Use the same X-ray wavelength for both reflections.  This is an ophyd Signal. Use its `.put()` method.

In [3]:
fourc.beam.wavelength.put(1.54)

### Specify the first reflection

Provide the set of angles that correspond with the reflection's Miller indices: (_hkl_)

The `setor()` (set orienting reflection) method uses the diffractometer's wavelength *at the time it is called*.  (To add reflections at different wavelengths, add a `wavelength=1.0` keyword argument with the correct value.)  

In [4]:
r1 = setor(4, 0, 0, tth=69.0966, omega=-145.451, chi=0, phi=0)

### Specify the second reflection

In [5]:
r2 = setor(0, 4, 0, tth=69.0966, omega=-145.451, chi=90, phi=0)

### Compute the *UB* orientation matrix

The `calc_UB()` method returns the computed **UB** matrix.

In [6]:
calc_UB(r1, r2)

[[-1.4134285e-05, -1.4134285e-05, -1.156906937382],
 [-1.156906937469, 1.73e-10, 1.4134285e-05],
 [0.0, 1.156906937469, -1.4134285e-05]]

## Report our setup

In [7]:
pa()

diffractometer='fourc'
HklSolver(name='hkl_soleil', version='5.1.2', geometry='E4CH', engine_name='hkl', mode='bissector')
Sample(name='silicon', lattice=Lattice(a=5.431, system='cubic'))
Reflection(name='r_0e75', h=4, k=0, l=0)
Reflection(name='r_2ab3', h=0, k=4, l=0)
Orienting reflections: ['r_0e75', 'r_2ab3']
U=[[-1.2217305e-05, -1.2217305e-05, -0.999999999851], [-0.999999999925, 1.49e-10, 1.2217305e-05], [0.0, 0.999999999925, -1.2217305e-05]]
UB=[[-1.4134285e-05, -1.4134285e-05, -1.156906937382], [-1.156906937469, 1.73e-10, 1.4134285e-05], [0.0, 1.156906937469, -1.4134285e-05]]
constraint: -180.0 <= omega <= 180.0
constraint: -180.0 <= chi <= 180.0
constraint: -180.0 <= phi <= 180.0
constraint: -180.0 <= tth <= 180.0
Mode: bissector
beam={'class': 'WavelengthXray', 'source_type': 'Synchrotron X-ray Source', 'energy': 8.050921976530415, 'wavelength': 1.54, 'energy_units': 'keV', 'wavelength_units': 'angstrom'}
h=0, k=0, l=0
omega=0, chi=0, phi=0, tth=0


## Check the orientation matrix

Perform checks with `forward()` (hkl to angle) and
`inverse()` (angle to hkl) computations to verify the diffractometer
will move to the same positions where the reflections were identified.

### Constrain the motors to limited ranges

* keep `tth` in the positive range
* keep `omega` in the negative range
* allow for slight roundoff errors
* keep `phi` fixed at zero

First, we apply constraints directly to the `calc`-level support.

In [8]:
fourc.core.constraints["tth"].limits = -0.001, 180
fourc.core.constraints["omega"].limits = (-180, 0.001)
fourc.core.constraints

['-180.0 <= omega <= 0.001', '-180.0 <= chi <= 180.0', '-180.0 <= phi <= 180.0', '-0.001 <= tth <= 180.0']

### (400) reflection test

1. Check the `inverse()` (angles -> (_hkl_)) computation.
1. Check the `forward()` ((_hkl_) -> angles) computation.

#### Check `inverse()` at (400)

To calculate the (_hkl_) corresponding to a given set of motor angles,
call `fourc.inverse()`.

The _hkl_ values are provided as a Python [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) structure.

In [9]:
fourc.inverse((-145.451, 0, 0, 69.0966))

Hklpy2DiffractometerPseudoPos(h=6.159871816956, k=0, l=0)

#### Check `forward(400)`

Compute the angles necessary to position the diffractometer
for the given reflection.

Note that for the forward computation, more than one set of angles may be used to reach the same crystal reflection.  This test will report the *default* selection.  The *default* selection (which may be changed through methods described in module :mod:`hklpy2.ops`) is the first solution.

function | returns
--- | ---
`cahkl()` | The *default* solution
`cahkl_table()` | Table of all allowed solutions.

Here we print the *default* solution (the one returned by calling 
`cahkl()`.  This command is a shortcut to `fourc.forward()`).

In [10]:
cahkl(4, 0, 0)

Hklpy2DiffractometerRealPos(omega=-158.391966399104, chi=0, phi=0.000700057668, tth=43.216067201793)

Show the table of all forward solutions for {math}`(4\ 0\ 0)` allowed by the current constraints.  Since this function accepts a *list* of {math}`hkl` reflections, extra Python syntax is applied.

In [11]:
cahkl_table((4, 0, 0))

======= = ======== ====== ========= =======
(hkl)   # omega    chi    phi       tth    
======= = ======== ====== ========= =======
(4 0 0) 1 -158.392 0      0.0007    43.2161
(4 0 0) 2 -21.608  0      -136.7832 43.2161
(4 0 0) 3 -21.608  -180.0 -43.2154  43.2161
(4 0 0) 4 -21.608  180.0  -43.2154  43.2161
(4 0 0) 5 -158.392 -180.0 -179.9993 43.2161
(4 0 0) 6 -158.392 180.0  -179.9993 43.2161
======= = ======== ====== ========= =======



### (040) reflection test

Repeat the `inverse` and `forward` calculations for the
second orientation reflection.

#### Check the inverse calculation: (040)

In [12]:
fourc.inverse(-145.451, 90, 0, 69.0966)

Hklpy2DiffractometerPseudoPos(h=9.19e-10, k=6.159871816956, l=0)

#### Check the forward calculation: (040)

In [13]:
fourc.forward(0, 4, 0)

Hklpy2DiffractometerRealPos(omega=-158.391966390296, chi=90.000699987971, phi=-89.996811077873, tth=43.216067219409)

## Scan in reciprocal space using Bluesky

To scan with Bluesky, we need more setup.

In [14]:
from bluesky import RunEngine
from bluesky import SupplementalData
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.plans as bp
import bluesky.plan_stubs as bps
import databroker

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2
sd = SupplementalData()

RE = RunEngine({})
RE.md = {}
RE.preprocessors.append(sd)
RE.subscribe(cat.v1.insert)
RE.subscribe(bec)

1

Setup the `RE` to save the `fourc` configuration with every run.

In [15]:
crw = hklpy2.ConfigurationRunWrapper(fourc)
RE.preprocessors.append(crw.wrapper)

### (_h00_) scan near (400)

In this example, we have no detector.  Still, we add the diffractometer
object in the detector list so that the _hkl_ and motor positions will appear
as columns in the table.

In [16]:
pos = fourc.forward(4, 0, 0)
pos

Hklpy2DiffractometerRealPos(omega=-158.39196637751, chi=-1.1e-11, phi=0.000700519158, tth=43.216067244979)

In [17]:
fourc.core.mode = "bissector"
fourc.move(4, 0, 0)
wh()

wavelength=1.54
h=4.0, k=0, l=0
omega=-34.5491, chi=0, phi=-110.9011, tth=69.0982


In [18]:
RE(bp.scan([fourc], fourc.h, 3.9, 4.1, fourc.k, 0, 0, fourc.l, 0, 0, 5))



Transient Scan ID: 1     Time: 2025-04-09 22:25:28
Persistent Unique Scan ID: '1c399e3b-df75-4b94-8000-fb6227cd5a28'
New stream: 'primary'
+-----------+------------+------------+------------+------------+-----------------------+-------------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k |    fourc_l | fourc_beam_wavelength | fourc_beam_energy | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+------------+------------+-----------------------+-------------------+-------------+------------+------------+------------+
|         1 | 22:25:28.7 |      3.900 |     -0.000 |      0.000 |                 1.540 |             8.051 |     -33.569 |      0.000 |   -112.862 |     67.137 |
|         2 | 22:25:28.7 |      3.950 |      0.000 |      0.000 |                 1.540 |             8.051 |     -34.057 |     -0.000 |   -111.884 |     68.115 |
|         3 | 22:25:28.7 |      4.000 |     

('1c399e3b-df75-4b94-8000-fb6227cd5a28',)

### chi scan from (400) to (040)

If we do this with $\omega=-145.4500$ and $2\theta=69.0985$, this will be a scan between the two orientation reflections.

Use `%mov` (IPython *magic* command) to move both motors at the same time.

In [19]:
# same as orientation reflections
RE(bps.mv(fourc.omega,-145.4500, fourc.tth,69.0985))

RE(bp.scan([fourc], fourc.chi, 0, 90, 10))



Transient Scan ID: 2     Time: 2025-04-09 22:25:28
Persistent Unique Scan ID: '4d7961a9-c191-4a6f-9386-6be0445ab985'
New stream: 'primary'
+-----------+------------+------------+-----------------------+-------------------+------------+------------+------------+-------------+------------+------------+
|   seq_num |       time |  fourc_chi | fourc_beam_wavelength | fourc_beam_energy |    fourc_h |    fourc_k |    fourc_l | fourc_omega |  fourc_phi |  fourc_tth |
+-----------+------------+------------+-----------------------+-------------------+------------+------------+------------+-------------+------------+------------+
|         1 | 22:25:28.9 |      0.000 |                 1.540 |             8.051 |     -1.297 |     -0.000 |     -3.784 |    -145.450 |   -108.917 |     69.099 |
|         2 | 22:25:28.9 |     10.000 |                 1.540 |             8.051 |     -1.277 |      0.695 |     -3.727 |    -145.450 |   -108.917 |     69.099 |
|         3 | 22:25:28.9 |     20.000 |     

('4d7961a9-c191-4a6f-9386-6be0445ab985',)

### (_0k0_) scan near (040)

In [20]:
RE(bp.scan([fourc], fourc.k, 3.9, 4.1, 5))



Transient Scan ID: 3     Time: 2025-04-09 22:25:29
Persistent Unique Scan ID: 'abb77f44-da59-41f1-9f41-a043798c164a'
New stream: 'primary'
+-----------+------------+------------+-----------------------+-------------------+------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_k | fourc_beam_wavelength | fourc_beam_energy |    fourc_h |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+-----------------------+-------------------+------------+------------+-------------+------------+------------+------------+
|         1 | 22:25:29.1 |      3.900 |                 1.540 |             8.051 |      4.100 |      0.000 |    -126.653 |    136.432 |   -179.999 |    106.695 |
|         2 | 22:25:29.1 |      3.950 |                 1.540 |             8.051 |      4.100 |      0.000 |    -126.180 |    136.067 |   -179.999 |    107.641 |
|         3 | 22:25:29.1 |      4.000 |     

('abb77f44-da59-41f1-9f41-a043798c164a',)

### (_hk0_) scan near (440)

In [21]:
RE(bp.scan([fourc], fourc.h, 3.9, 4.1, fourc.k, 3.9, 4.1, 5))



Transient Scan ID: 4     Time: 2025-04-09 22:25:29
Persistent Unique Scan ID: 'e7abe651-e275-44eb-b0be-c0e67bb3b8b3'
New stream: 'primary'
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k | fourc_beam_wavelength | fourc_beam_energy |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|         1 | 22:25:29.3 |      3.900 |      3.900 |                 1.540 |             8.051 |     -0.000 |    -128.559 |    135.000 |   -179.999 |    102.882 |
|         2 | 22:25:29.3 |      3.950 |      3.950 |                 1.540 |             8.051 |      0.000 |    -127.628 |    135.000 |   -179.999 |    104.744 |
|         3 | 22:25:29.3 |      4.000 |     

('e7abe651-e275-44eb-b0be-c0e67bb3b8b3',)

Move to the (_440_) reflection.

In [22]:
fourc.move((4,4,0))
print(f"{fourc.position = }")

fourc.position = Hklpy2DiffractometerPseudoPos(h=3.999999991917, k=3.999999998121, l=-3.289e-09)


Repeat the same scan about the (_440_) but use _relative_ positions.

In [23]:
RE(bp.rel_scan([fourc], fourc.h, -0.1, 0.1, fourc.k, -0.1, 0.1, 5))



Transient Scan ID: 5     Time: 2025-04-09 22:25:29
Persistent Unique Scan ID: 'b96baadc-8961-4994-b3c8-153f5ed15392'
New stream: 'primary'
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |    fourc_h |    fourc_k | fourc_beam_wavelength | fourc_beam_energy |    fourc_l | fourc_omega |  fourc_chi |  fourc_phi |  fourc_tth |
+-----------+------------+------------+------------+-----------------------+-------------------+------------+-------------+------------+------------+------------+
|         1 | 22:25:29.5 |      3.900 |      3.900 |                 1.540 |             8.051 |     -0.000 |    -128.559 |    135.000 |   -179.999 |    102.882 |
|         2 | 22:25:29.5 |      3.950 |      3.950 |                 1.540 |             8.051 |     -0.000 |    -127.628 |    135.000 |   -179.999 |    104.744 |
|         3 | 22:25:29.6 |      4.000 |     

('b96baadc-8961-4994-b3c8-153f5ed15392',)

### Show the configuration

Print the diffractometer configuration that was saved with the run.

In [24]:
# cat.v2[-1].start["diffractometers"]["fourc"]
cat.v2[-1].metadata["start"]["diffractometers"]["fourc"]

{'_header': {'datetime': '2025-04-09 22:25:29.552005',
  'hklpy2_version': '0.0.29.dev43+gdc399a0.d20250410',
  'python_class': 'Hklpy2Diffractometer'},
 'name': 'fourc',
 'axes': {'pseudo_axes': ['h', 'k', 'l'],
  'real_axes': ['omega', 'chi', 'phi', 'tth'],
  'axes_xref': {'h': 'h',
   'k': 'k',
   'l': 'l',
   'omega': 'omega',
   'chi': 'chi',
   'phi': 'phi',
   'tth': 'tth'},
  'extra_axes': {'h2': 0, 'k2': 0, 'l2': 0, 'psi': 0}},
 'sample_name': 'silicon',
 'samples': {'sample': {'name': 'sample',
   'lattice': {'a': 1,
    'b': 1,
    'c': 1,
    'alpha': 90.0,
    'beta': 90.0,
    'gamma': 90.0},
   'reflections': {},
   'reflections_order': [],
   'U': [[1, 0, 0], [0, 1, 0], [0, 0, 1]],
   'UB': [[6.283185307179586, 0.0, 0.0],
    [0.0, 6.283185307179586, 0.0],
    [0.0, 0.0, 6.283185307179586]],
   'digits': 4},
  'silicon': {'name': 'silicon',
   'lattice': {'a': 5.431020511,
    'b': 5.431020511,
    'c': 5.431020511,
    'alpha': 90,
    'beta': 90,
    'gamma': 90},
   